In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("completed_movie_dataset.csv")

In [3]:
dataset.head(5)

,imdb_id,movie_title,genre,imdb_rating,metascore,vote_count,release_year,release_month,runtime,actors,director,budget,worldwide_gross,origin_Language,origin_Country
0,tt0172495,Gladiator,action,8.5,67.0,1743461,2000,5.0,9300.0,"['Russell Crowe', 'Joaquin Phoenix', 'Connie N...",Ridley Scott,"$103,000,000 (estimated)","$465,516,248",English,United States
1,tt0212346,Miss Congeniality,action,6.3,43.0,238741,2000,12.0,6540.0,"['Sandra Bullock', 'Michael Caine', 'Benjamin ...",Donald Petrie,"$45,000,000 (estimated)","$212,742,720",English,United States
2,tt0187393,The Patriot,action,7.2,63.0,302532,2000,6.0,9900.0,"['Mel Gibson', 'Heath Ledger', 'Joely Richards...",Roland Emmerich,"$110,000,000 (estimated)","$215,294,342",English,Germany
3,tt0120755,Mission: Impossible II,action,6.1,59.0,389300,2000,5.0,7380.0,"['Tom Cruise', 'Dougray Scott', 'Thandiwe Newt...",John Woo,"$125,000,000 (estimated)","$546,388,108",English,United States
4,tt0187078,Gone in Sixty Seconds,action,6.5,35.0,301954,2000,6.0,7080.0,"['Nicolas Cage', 'Angelina Jolie', 'Giovanni R...",Dominic Sena,"$90,000,000 (estimated)","$237,202,299",English,United States


In [4]:
# Understanding the size of the dataset
dataset.shape

(6666, 15)

In [5]:
# Understanding the columns of the dataset
dataset.columns

Index(['imdb_id', 'movie_title', 'genre', 'imdb_rating', 'metascore',
       'vote_count', 'release_year', 'release_month', 'runtime', 'actors',
       'director', 'budget', 'worldwide_gross', 'origin_Language',
       'origin_Country'],
      dtype='object')

**Handling Duplicated Data**

In [6]:
# Checking for duplicate data

duplicates = dataset[dataset.duplicated(subset='imdb_id', keep=False)]
duplicates.shape

(0, 15)

**Handling Null Values**

In [7]:
# Check for null values

dataset.isnull().sum()

imdb_id               0
movie_title           0
genre                 0
imdb_rating           0
metascore          2485
vote_count            0
release_year          0
release_month       111
runtime              65
actors                0
director            137
budget             4483
worldwide_gross    2117
origin_Language      40
origin_Country       19
dtype: int64

In [8]:
# Understandign the missing value percentage of each column

dataset.isnull().mean() *100

imdb_id             0.000000
movie_title         0.000000
genre               0.000000
imdb_rating         0.000000
metascore          37.278728
vote_count          0.000000
release_year        0.000000
release_month       1.665167
runtime             0.975098
actors              0.000000
director            2.055206
budget             67.251725
worldwide_gross    31.758176
origin_Language     0.600060
origin_Country      0.285029
dtype: float64

- Budeget columnd has more than 50% of missing values, which make it difficult to draw accurate conclusions.
  Therefore it is suitable to drop the entire column.

- Release month, origin_langauge and origin_country are categorical variables, where we can use the mode to impute the missing values

- Director column's missing values can be imputed with Unknown placehodler

- Runtime missing values can be imputed with mean value

- Worldwide gross has 32% missing values and these can be imputed using the mean.
  However, worldwide gross can vastly vary based on the genre of the movie.

In [9]:
# Budget is missing in more than 50% of rows. Since this is too high column is dropped
dataset.drop(columns=['budget'], inplace=True)

In [10]:
# Filling categorical columns with Unknown placeholder
dataset['director'].fillna('Unknown', inplace=True)

In [11]:
# Fill missing month with the most common month
dataset['release_month'].fillna(dataset['release_month'].mode()[0], inplace=True)

In [12]:
# Fill missing language and country with most common values
dataset['origin_Language'].fillna(dataset['origin_Language'].mode()[0], inplace=True)
dataset['origin_Country'].fillna(dataset['origin_Country'].mode()[0], inplace=True)

In [13]:
# Filling runtime with the mean value
dataset['runtime'].fillna(dataset['runtime'].median(), inplace=True)

Cleaning Worldwide Gross column's values

In [14]:
# In this column $ and , should be removed and values should be converted to numeric type

# Removing $ prefix and . 
dataset["worldwide_gross"] = dataset["worldwide_gross"].str.replace("$", "")
dataset["worldwide_gross"] = dataset["worldwide_gross"].str.replace(",", "")

# Converting to numeric
dataset['worldwide_gross'] = pd.to_numeric(dataset['worldwide_gross'], errors='coerce')

Understanding the behaviours of null values in worldwide gross

In [15]:
gross_null_df = dataset[dataset['worldwide_gross'].isna()]

In [16]:
# Genre percentages of missing values of worldwide gross

genre_percentages = gross_null_df['genre'].value_counts(normalize=True) * 100

# Display the result
print(genre_percentages)

genre
musical      30.089750
animation    17.997166
horror       11.761927
family        9.400094
fantasy       6.613132
crime         5.196032
action        4.204062
adventure     4.204062
mystery       3.967879
romance       3.353803
comedy        2.834199
thriller      0.377893
Name: proportion, dtype: float64


In [23]:
# Mean gross of each genre
gross_non_null_df = dataset[~dataset['worldwide_gross'].isna()]

genre_mean_gross = gross_non_null_df.groupby('genre')['worldwide_gross'].mean()
genre_mean_gross_sorted = genre_mean_gross.sort_values(ascending=False)
print(genre_mean_gross_sorted)

genre
action       1.603897e+08
adventure    1.437131e+08
comedy       6.846427e+07
thriller     4.828143e+07
horror       3.445797e+07
crime        2.988725e+07
romance      2.703178e+07
mystery      2.485419e+07
family       2.231375e+07
fantasy      1.939231e+07
animation    1.878610e+07
musical      7.084917e+06
Name: worldwide_gross, dtype: float64


In [25]:
print(gross_non_null_df['worldwide_gross'].mean())

70517737.58452407


We can see from above analysis that mean worldwide gross according to genre is vastly different.
We can also see that mean of worldwide gross is also way higher than the mean gross of musical, animation, horror and family columns.
These columns have the highest percentages of missing values in worldwide gross column

Therefore, I will be imputing worldwide gross column with the mean gross of respective genre

In [26]:
# Imputing worldwide_gross with mean of respective genre
genre_mean_dict = dataset.groupby('genre')['worldwide_gross'].mean().to_dict()

def impute_worldwide_gross(row):
    if pd.isna(row['worldwide_gross']):  # Check if worldwide_gross is missing
        return genre_mean_dict.get(row['genre'], np.nan)  # Use genre mean or NaN if genre not found
    return row['worldwide_gross']  # Return original value if not missing


dataset['worldwide_gross'] = dataset.apply(impute_worldwide_gross, axis=1)

In [27]:
dataset.isnull().sum()

imdb_id               0
movie_title           0
genre                 0
imdb_rating           0
metascore          2485
vote_count            0
release_year          0
release_month         0
runtime               0
actors                0
director              0
worldwide_gross       0
origin_Language       0
origin_Country        0
dtype: int64

In [28]:
# Now we have to handle the missing values in metascore

In [29]:
meta_score_df = dataset[dataset['metascore'].isna()]

In [30]:
# Genre percentages of the missing metascore rows

genre_percentages = meta_score_df['genre'].value_counts(normalize=True) * 100
print(genre_percentages)

genre
musical      29.859155
animation    23.822938
family       10.181087
horror        9.778672
fantasy       6.599598
adventure     4.828974
action        3.702213
mystery       3.661972
crime         3.460765
romance       2.535211
comedy        1.368209
thriller      0.201207
Name: proportion, dtype: float64


In [32]:
country_percentages = meta_score_df['origin_Country'].value_counts(normalize=True) * 100

# Display the result
print(country_percentages)

origin_Country
United States                     29.657948
Japan                             12.354125
India                             12.072435
United Kingdom                     6.961771
Germany                            4.104628
                                    ...    
Colombia                           0.040241
Israel                             0.040241
Syria                              0.040241
Federal Republic of Yugoslavia     0.040241
Iraq                               0.040241
Name: proportion, Length: 74, dtype: float64


In [35]:
non_null_meta_score_df = dataset[~dataset['metascore'].isna()]

non_null_country_percentages = non_null_meta_score_df['origin_Country'].value_counts(normalize=True) * 100

# Display the result
print(non_null_country_percentages)

origin_Country
United States            63.764650
United Kingdom           12.006697
France                    4.066013
Canada                    3.491988
Germany                   2.965798
Japan                     2.224348
Australia                 1.411146
Spain                     1.100215
South Korea               0.837120
Ireland                   0.837120
China                     0.741449
Hong Kong                 0.550108
Italy                     0.526190
India                     0.406601
New Zealand               0.382684
Sweden                    0.382684
Czech Republic            0.358766
Denmark                   0.358766
Mexico                    0.358766
Belgium                   0.310930
Russia                    0.239177
Norway                    0.215260
Argentina                 0.215260
Switzerland               0.191342
Brazil                    0.191342
Netherlands               0.167424
United Arab Emirates      0.143506
South Africa              0.143506
Hunga

We can see from above analysis that 65% of missing metascore values are from countries outside of the USA and UK
However, in avalibale metascores 76% of values are from the UK and USA.
Which means that in countries outside of UK and USA, less people have rated the movies.
Moreover, similar insights can be taken from the IMDB ratings where there are no null values.
Therefore, I will be dropping the metascore column

In [36]:
dataset.drop(columns=['metascore'], inplace=True)

**Handling Data Types**

In [37]:
# Convert worldwide_gross to integers
dataset['worldwide_gross'] = dataset['worldwide_gross'].astype(int)
# Convert month into int type
dataset['release_month'] = dataset['release_month'].astype(int)
# Convert runtime from seconds into minutes
dataset['runtime'] = dataset['runtime']/60
# Convert runtime into int
dataset['runtime'] = dataset['runtime'].astype(int)

In [38]:
dataset.head(5)

,imdb_id,movie_title,genre,imdb_rating,vote_count,release_year,release_month,runtime,actors,director,worldwide_gross,origin_Language,origin_Country
0,tt0172495,Gladiator,action,8.5,1743461,2000,5,155,"['Russell Crowe', 'Joaquin Phoenix', 'Connie N...",Ridley Scott,465516248,English,United States
1,tt0212346,Miss Congeniality,action,6.3,238741,2000,12,109,"['Sandra Bullock', 'Michael Caine', 'Benjamin ...",Donald Petrie,212742720,English,United States
2,tt0187393,The Patriot,action,7.2,302532,2000,6,165,"['Mel Gibson', 'Heath Ledger', 'Joely Richards...",Roland Emmerich,215294342,English,Germany
3,tt0120755,Mission: Impossible II,action,6.1,389300,2000,5,123,"['Tom Cruise', 'Dougray Scott', 'Thandiwe Newt...",John Woo,546388108,English,United States
4,tt0187078,Gone in Sixty Seconds,action,6.5,301954,2000,6,118,"['Nicolas Cage', 'Angelina Jolie', 'Giovanni R...",Dominic Sena,237202299,English,United States


In [39]:
# Saving the preprocessed dataset

dataset.to_csv("preprocessed_dataset.csv", index = False)